##**1.Load Data from kaggle:**

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d rhuebner/human-resources-data-set


  0% 0.00/16.6k [00:00<?, ?B/s]
100% 16.6k/16.6k [00:00<00:00, 37.1MB/s]


In [5]:
!unzip -q human-resources-data-set.zip -d /content/Data

##**1.EDA:**

In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [7]:
data = pd.read_csv('/content/Data/HRDataset_v14.csv')

In [8]:
data.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,...,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,...,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,...,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,...,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,...,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

In [ ]:
!pip install pandas-profiling

In [11]:
from ydata_profiling import ProfileReport

In [12]:
profile = ProfileReport(data, title="Pandas Profiling Report")

In [ ]:
profile.to_notebook_iframe()

In [14]:
# List of columns to drop
columns_to_drop = ['Employee_Name', 'EmpID', 'ManagerName', 'ManagerID', 'DateofHire', 'DateofTermination','LastPerformanceReview_Date','DOB']

# Drop the specified columns
data = data.drop(columns=columns_to_drop, axis=1)

In [15]:
# Separate numerical and categorical columns
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = data.select_dtypes(include=['object']).columns

# Create DataFrames for numerical and categorical columns
numerical_df = data[numerical_columns]
categorical_df = data[categorical_columns]

In [16]:
numerical_df.head()

,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Zip,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
0,0,0,1,1,5,4,0,62506,0,19,1960,4.60,5,0,0,1
1,1,1,1,5,3,3,0,104437,1,27,2148,4.96,3,6,0,17
2,1,1,0,5,5,3,0,64955,1,20,1810,3.02,3,0,0,3
3,1,1,0,1,5,3,0,64991,0,19,1886,4.84,5,0,0,15
4,0,2,0,5,5,3,0,50825,1,19,2169,5.00,4,0,0,2


In [17]:
categorical_df.head()

,Position,State,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,TermReason,EmploymentStatus,Department,RecruitmentSource,PerformanceScore
0,Production Technician I,MA,M,Single,US Citizen,No,White,N/A-StillEmployed,Active,Production,LinkedIn,Exceeds
1,Sr. DBA,MA,M,Married,US Citizen,No,White,career change,Voluntarily Terminated,IT/IS,Indeed,Fully Meets
2,Production Technician II,MA,F,Married,US Citizen,No,White,hours,Voluntarily Terminated,Production,LinkedIn,Fully Meets
3,Production Technician I,MA,F,Married,US Citizen,No,White,N/A-StillEmployed,Active,Production,Indeed,Fully Meets
4,Production Technician I,MA,F,Divorced,US Citizen,No,White,return to school,Voluntarily Terminated,Production,Google Search,Fully Meets


In [18]:
# Get unique values for each categorical column
unique_values_dict = {}
for column in categorical_df.columns:
    unique_values_dict[column] = categorical_df[column].unique()

print("Unique values for each categorical column:")
for column, unique_values in unique_values_dict.items():
    print(f"{column}: {unique_values}")

Unique values for each categorical column:
Position: ['Production Technician I' 'Sr. DBA' 'Production Technician II'
 'Software Engineer' 'IT Support' 'Data Analyst' 'Database Administrator'
 'Enterprise Architect' 'Sr. Accountant' 'Production Manager'
 'Accountant I' 'Area Sales Manager' 'Software Engineering Manager'
 'BI Director' 'Director of Operations' 'Sr. Network Engineer'
 'Sales Manager' 'BI Developer' 'IT Manager - Support' 'Network Engineer'
 'IT Director' 'Director of Sales' 'Administrative Assistant'
 'President & CEO' 'Senior BI Developer' 'Shared Services Manager'
 'IT Manager - Infra' 'Principal Data Architect' 'Data Architect'
 'IT Manager - DB' 'Data Analyst ' 'CIO']
State: ['MA' 'TX' 'CT' 'VA' 'VT' 'AL' 'WA' 'CA' 'OH' 'IN' 'TN' 'NH' 'RI' 'PA'
 'CO' 'NY' 'UT' 'GA' 'FL' 'NC' 'KY' 'ID' 'NV' 'MT' 'OR' 'ND' 'AZ' 'ME']
Sex: ['M ' 'F']
MaritalDesc: ['Single' 'Married' 'Divorced' 'Widowed' 'Separated']
CitizenDesc: ['US Citizen' 'Eligible NonCitizen' 'Non-Citizen']
Hispanic

In [19]:
# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

print("Data after label encoding:")
data.head()

Data after label encoding:


,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,...,TermReason,EmploymentStatus,Department,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
0,0,0,1,1,5,4,0,62506,0,19,...,3,0,3,5,0,4.60,5,0,0,1
1,1,1,1,5,3,3,0,104437,1,27,...,5,2,2,4,1,4.96,3,6,0,17
2,1,1,0,5,5,3,0,64955,1,20,...,7,2,3,5,1,3.02,3,0,0,3
3,1,1,0,1,5,3,0,64991,0,19,...,3,0,3,4,1,4.84,5,0,0,15
4,0,2,0,5,5,3,0,50825,1,19,...,16,2,3,3,1,5.00,4,0,0,2


In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Assuming your data is already loaded into a DataFrame named 'data'

# Separate features and target variable
X = data.iloc[:, :-1]
y = data['Termd']  # 'Termd' is now the binary classification target

# Initialize classification models
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    SVC(),
    KNeighborsClassifier(),
    MLPClassifier(),
]

# Loop through models and fit them
for model in models:
    model_name = model.__class__.__name__
    print(f"Training {model_name}...")

    # Perform cross-validation and get predicted labels
    y_pred = cross_val_predict(model, X, y, cv=5)

    # Print classification report
    report = classification_report(y, y_pred)
    print(f"{model_name} Classification Report:")
    print(report)
    print("="*40)


Training LogisticRegression...
LogisticRegression Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       207
           1       0.98      0.96      0.97       104

    accuracy                           0.98       311
   macro avg       0.98      0.98      0.98       311
weighted avg       0.98      0.98      0.98       311

Training DecisionTreeClassifier...
DecisionTreeClassifier Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       207
           1       1.00      1.00      1.00       104

    accuracy                           1.00       311
   macro avg       1.00      1.00      1.00       311
weighted avg       1.00      1.00      1.00       311

Training RandomForestClassifier...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

RandomForestClassifier Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       207
           1       1.00      1.00      1.00       104

    accuracy                           1.00       311
   macro avg       1.00      1.00      1.00       311
weighted avg       1.00      1.00      1.00       311

Training GradientBoostingClassifier...
GradientBoostingClassifier Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       207
           1       1.00      1.00      1.00       104

    accuracy                           1.00       311
   macro avg       1.00      1.00      1.00       311
weighted avg       1.00      1.00      1.00       311

Training SVC...
SVC Classification Report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80       207
           1       0.00      0.00      0.00       104

    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MLPClassifier Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.80      0.72       207
           1       0.32      0.19      0.24       104

    accuracy                           0.59       311
   macro avg       0.49      0.49      0.48       311
weighted avg       0.55      0.59      0.56       311

